**IMPORTING DATA**

In [1]:
import yfinance as yf

# Define the ticker symbol for S&P 500 (SPY is the ETF that tracks it)
ticker_symbol = "SPY"

# Define the start and end dates for your data
start_date = "2000-12-01"
end_date = "2023-09-01"

# Fetch historical S&P 500 data from Yahoo Finance
sp500_data = yf.download(ticker_symbol, start=start_date, end=end_date)

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()
[*********************100%%**********************]  1 of 1 completed


In [2]:
pip install fredapi


In [13]:
import os
from fredapi import Fred

# Retrieve API key from environment variable
api_key = os.environ.get('FRED_API_KEY')

# Initialize the FRED API client with your API key
fred = Fred(api_key=api_key)

# Define the series IDs for the economic indicators you want to retrieve
inflation_series_id = 'CPIAUCSL'  # Consumer Price Index for All Urban Consumers (CPI-U)
interest_rate_series_id = 'FEDFUNDS'  # Effective Federal Funds Rate
labor_openings_series_id = 'JTSJOL'  # Job Openings

# Fetch economic indicator data from FRED
inflation_data = fred.get_series(inflation_series_id, start=start_date, end=end_date)
interest_rate_data = fred.get_series(interest_rate_series_id, start=start_date, end=end_date)
labor_openings_data = fred.get_series(labor_openings_series_id, start=start_date, end=end_date)


ValueError: You need to set a valid API key. You can set it in 3 ways:
pass the string with api_key, or set api_key_file to a
file with the api key in the first line, or set the
environment variable 'FRED_API_KEY' to the value of your
api key. You can sign up for a free api key on the Fred
website at http://research.stlouisfed.org/fred2/

In [ ]:
inflation_data.head(100)

In [ ]:
interest_rate_data.head(100)

In [ ]:
labor_openings_data.head(100)

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and subplots for S&P 500 and economic data
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))

# Plot S&P 500 data
axes[0, 0].plot(sp500_data.index, sp500_data['Close'], label='S&P 500', color='blue')
axes[0, 0].set_title('S&P 500')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Price')

# Plot inflation data
axes[0, 1].plot(inflation_data.index, inflation_data, label='Inflation Rate', color='green')
axes[0, 1].set_title('Inflation Rate')
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Rate')

# Plot interest rate data
axes[1, 0].plot(interest_rate_data.index, interest_rate_data, label='Interest Rate', color='red')
axes[1, 0].set_title('Interest Rate')
axes[1, 0].set_xlabel('Date')
axes[1, 0].set_ylabel('Rate')

# Plot labor openings data
axes[1, 1].plot(labor_openings_data.index, labor_openings_data, label='Labor Openings', color='purple')
axes[1, 1].set_title('Labor Openings')
axes[1, 1].set_xlabel('Date')
axes[1, 1].set_ylabel('Number of Openings')

# Adjust layout and display legends
plt.tight_layout()
plt.legend()

# Show the plots
plt.show()


In [ ]:
import pandas as pd

# Convert the economic data Series to DataFrames with appropriate column names
inflation_df = pd.DataFrame({'Inflation_Rate': inflation_data})
interest_rate_df = pd.DataFrame({'Interest_Rate': interest_rate_data})
labor_openings_df = pd.DataFrame({'Labor_Openings': labor_openings_data})

# Merge the DataFrames with your S&P 500 data
merged_data = sp500_data.join([inflation_df, interest_rate_df, labor_openings_df], how='inner')


In [ ]:
import seaborn as sns

# Create a DataFrame containing all the data you want to analyze
# In this case, we'll use the merged_data DataFrame
data_to_analyze = merged_data[['Close', 'Inflation_Rate', 'Interest_Rate', 'Labor_Openings']]

# Calculate the correlation matrix
correlation_matrix = data_to_analyze.corr()

# Create a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()


In [ ]:
import pandas as pd

In [ ]:
# Convert the economic data Series to DataFrames with appropriate column names
inflation_df = pd.DataFrame({'Inflation_Rate': inflation_data})
interest_rate_df = pd.DataFrame({'Interest_Rate': interest_rate_data})
labor_openings_df = pd.DataFrame({'Labor_Openings': labor_openings_data})

# Merge the DataFrames with your S&P 500 data
merged_data = sp500_data.join([inflation_df, interest_rate_df, labor_openings_df], how='inner')

In [ ]:
# Calculate 7-day rolling mean and standard deviation of S&P 500 prices
merged_data['SMA_7'] = merged_data['Close'].rolling(window=7).mean()
merged_data['STD_7'] = merged_data['Close'].rolling(window=7).std()

# Calculate percentage change in S&P 500 prices
merged_data['Price_Change'] = merged_data['Close'].pct_change()

In [ ]:
merged_data = merged_data.iloc[6:]

In [ ]:
merged_data.head(100)

In [ ]:
from sklearn.model_selection import train_test_split

# Define your features and target variable
X = merged_data[['SMA_7', 'STD_7', 'Inflation_Rate', 'Interest_Rate', 'Labor_Openings']]
y = merged_data['Price_Change']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)


**LINEAR REGRESSION MODEL**

In [ ]:
import numpy as np
import random
import tensorflow as tf

# Set a random seed for NumPy
np.random.seed(42)

# Set a random seed for Python's random module
random.seed(42)


In [ ]:
from sklearn.linear_model import LinearRegression

# Create a Linear Regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

In [ ]:
# Predict the price change for the test data
lr_predictions = model.predict(X_test)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate MAE and MSE
lr_mae = mean_absolute_error(y_test, lr_predictions)
lr_mse = mean_squared_error(y_test, lr_predictions)

print(f'lr_MAE: {lr_mae}')
print(f'lr_MSE: {lr_mse}')


In [ ]:
# Define a threshold for predictions (e.g., 0.01 for a small price change)
threshold = 0.01

# Calculate the direction of the price change based on the threshold
y_test_direction = np.where(y_test > threshold, 1, np.where(y_test < -threshold, -1, 0))
predictions_direction = np.where(lr_predictions > threshold, 1, np.where(lr_predictions < -threshold, -1, 0))

# Calculate the precision and accuracy
lr_precision = np.sum((y_test_direction == 1) & (predictions_direction == 1)) / np.sum(predictions_direction == 1)
lr_accuracy = np.sum(y_test_direction == predictions_direction) / len(predictions_direction)

# Calculate the Z-score of the accuracy
n = len(predictions_direction)
lr_z_score = (lr_accuracy - 0.5) / np.sqrt((0.5 * (1 - 0.5)) / n)

print(f'lr_Precision: {lr_precision}')
print(f'lr_Accuracy: {lr_accuracy}')
print(f'lr_Z-score: {lr_z_score}')


**RANDOM FOREST MODEL**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Create a Random Forests model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model to the training data
rf_model.fit(X_train, y_train)

# Predict the price change for the test data
rf_predictions = rf_model.predict(X_test)

# Calculate MAE and MSE for Random Forests
rf_mae = mean_absolute_error(y_test, rf_predictions)
rf_mse = mean_squared_error(y_test, rf_predictions)

print("Random Forests Model:")
print(f'rf_MAE: {rf_mae}')
print(f'rf_MSE: {rf_mse}')


In [ ]:
# Define a threshold for predictions (e.g., 0.01 for a small price change)
threshold = 0.01

# Calculate the direction of the price change based on the threshold
y_test_direction = np.where(y_test > threshold, 1, np.where(y_test < -threshold, -1, 0))
predictions_direction = np.where(rf_predictions > threshold, 1, np.where(rf_predictions < -threshold, -1, 0))

# Calculate the precision and accuracy
rf_precision = np.sum((y_test_direction == 1) & (predictions_direction == 1)) / np.sum(predictions_direction == 1)
rf_accuracy = np.sum(y_test_direction == predictions_direction) / len(predictions_direction)

# Calculate the Z-score of the accuracy
n = len(predictions_direction)
rf_z_score = (rf_accuracy - 0.5) / np.sqrt((0.5 * (1 - 0.5)) / n)

print(f'rf_Precision: {rf_precision}')
print(f'rf_Accuracy: {rf_accuracy}')
print(f'rf_Z-score: {rf_z_score}')

**LONG SHORT-TERM MEMORY MODEL**

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Reshape data for LSTM (samples, time steps, features)
X_train_lstm = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_lstm = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

# Create an LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')

# Fit the LSTM model to the training data
lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, verbose=0)

# Predict the price change for the test data using the LSTM model
lstm_predictions = lstm_model.predict(X_test_lstm).flatten()

# Calculate MAE and MSE for LSTM
lstm_mae = mean_absolute_error(y_test, lstm_predictions)
lstm_mse = mean_squared_error(y_test, lstm_predictions)

print("LSTM Model:")
print(f'lstm_MAE: {lstm_mae}')
print(f'lstm_MSE: {lstm_mse}')


In [ ]:
# Define a threshold for predictions (e.g., 0.01 for a small price change)
threshold = 0.01

# Calculate the direction of the price change based on the threshold
y_test_direction = np.where(y_test > threshold, 1, np.where(y_test < -threshold, -1, 0))
predictions_direction = np.where(lstm_predictions > threshold, 1, np.where(lstm_predictions < -threshold, -1, 0))

# Calculate the precision and accuracy
lstm_precision = np.sum((y_test_direction == 1) & (predictions_direction == 1)) / np.sum(predictions_direction == 1)
lstm_accuracy = np.sum(y_test_direction == predictions_direction) / len(predictions_direction)

# Calculate the Z-score of the accuracy
lstm_n = len(predictions_direction)
lstm_z_score = (lstm_accuracy - 0.5) / np.sqrt((0.5 * (1 - 0.5)) / n)

print(f'lstm_Precision: {lstm_precision}')
print(f'lstm_Accuracy: {lstm_accuracy}')
print(f'lstm_Z-score: {lstm_z_score}')

In [ ]:
# Create a dictionary to store the scores for each model
scores = {
    'Model': ['Linear Regression', 'Random Forests', 'LSTM'],
    'MAE': [lr_mae, rf_mae, lstm_mae],
    'MSE': [lr_mse, rf_mse, lstm_mse],
    'Z-Score': [lr_z_score, rf_z_score, lstm_z_score],
    'Accuracy': [lr_accuracy, rf_accuracy, lstm_accuracy],
    'Precision': [lr_precision, rf_precision, lstm_precision]
}

# Create a DataFrame from the scores dictionary
scores_df = pd.DataFrame(scores)

# Display the comparison table
print(scores_df)

In [ ]:
# Data for 0.8 fit sampling
data_0p8 = {
    'Model': ['Linear Regression', 'Random Forests', 'LSTM'],
    'MAE': [0.044526, 0.043300, 0.048092],
    'MSE': [0.003572, 0.003560, 0.003439],
    'Z-Score': [-2.057983, -0.685994, -4.801960],
    'Accuracy': [0.411765, 0.470588, 0.294118],
    'Precision': [0.537313, 0.587500, None]
}

# Data for 0.5 fit sampling
data_0p5 = {
    'Model': ['Linear Regression', 'Random Forests', 'LSTM'],
    'MAE': [0.037967, 0.040806, 0.036507],
    'MSE': [0.002284, 0.002798, 0.002158],
    'Z-Score': [-2.494700, -1.626978, 0.325396],
    'Accuracy': [0.364706, 0.411765, 0.517647],
    'Precision': [0.536585, 0.543478, 0.517647]
}

# Data for 0.2 fit sampling
data_0p2 = {
    'Model': ['Linear Regression', 'Random Forests', 'LSTM'],
    'MAE': [0.038565, 0.049855, 0.036391],
    'MSE': [0.002421, 0.003780, 0.002001],
    'Z-Score': [-1.028992, -2.057983, -3.429972],
    'Accuracy': [0.411765, 0.323529, 0.205882],
    'Precision': [0.473684, 0.450000, None]
}

# Create DataFrames for each fit sampling size
df_0p8 = pd.DataFrame(data_0p8)
df_0p5 = pd.DataFrame(data_0p5)
df_0p2 = pd.DataFrame(data_0p2)

# Display the comparison tables
print("\nFitting 0.8:")
print(df_0p8)

print("\nFitting 0.5:")
print(df_0p5)

print("\nFitting 0.2:")
print(df_0p2)


**PLOTTING RESULTS**

LINEAR REGRESSION

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score


In [ ]:
# Set a random seed for NumPy
np.random.seed(42)

# Set a random seed for Python's random module
random.seed(42)

# Define the fitting values from 0.1 to 0.8 with a step of 0.1
fitting_values = np.arange(0.1, 0.9, 0.1)

# Initialize empty lists to store the differences in scores
lr_mae_differences = []
lr_mse_differences = []
lr_z_score_differences = []
lr_accuracy_differences = []
lr_precision_differences = []
lr_recall_differences = []

# Loop through fitting values and calculate scores
for i in range(len(fitting_values) - 1):
    # Define the fitting value for this iteration
    fitting_value = fitting_values[i]

    # Split the data into training and testing sets with the current fitting value
    X_train_sampled, X_test_sampled, y_train_sampled, y_test_sampled = train_test_split(
        X, y, test_size=fitting_value, random_state=42
    )

    # Create a Linear Regression lr_model
    lr_model = LinearRegression()

    # Fit the lr_model to the training data
    lr_model.fit(X_train_sampled, y_train_sampled)

    # Predict the price change for the test data
    lr_predictions = lr_model.predict(X_test_sampled)

    # Define a threshold for predictions (e.g., 0.01 for a small price change)
    threshold = 0.01

    # Calculate the direction of the price change based on the threshold
    y_test_direction = np.where(y_test_sampled > threshold, 1, np.where(y_test_sampled < -threshold, -1, 0))
    lr_predictions_direction = np.where(lr_predictions > threshold, 1, np.where(lr_predictions < -threshold, -1, 0))

    # Calculate MAE and MSE for the lr_predictions
    lr_mae_score = mean_absolute_error(y_test_sampled, lr_predictions)
    lr_mse_score = mean_squared_error(y_test_sampled, lr_predictions)

    # Calculate Z-score, accuracy, and precision
    lr_z_score = (accuracy_score(y_test_direction, lr_predictions_direction) - 0.5) / np.sqrt(
        (0.5 * (1 - 0.5)) / len(y_test_sampled)
    )
    lr_accuracy = accuracy_score(y_test_direction, lr_predictions_direction)
    lr_precision = precision_score(y_test_direction, lr_predictions_direction, average='micro')  # Use binary average
    lr_recall = recall_score(y_test_direction, lr_predictions_direction, average='micro')

    # Append the differences to the lists
    lr_mae_differences.append(lr_mae_score)
    lr_mse_differences.append(lr_mse_score)
    lr_z_score_differences.append(lr_z_score)
    lr_accuracy_differences.append(lr_accuracy)
    lr_precision_differences.append(lr_precision)
    lr_recall_differences.append(lr_recall)

# Create a line chart to visualize the differences with colorblind-friendly colors
plt.figure(figsize=(12, 8))
plt.plot(fitting_values[:-1], lr_mae_differences, label='MAE Difference', marker='o', color='tab:cyan')
plt.plot(fitting_values[:-1], lr_mse_differences, label='MSE Difference', marker='o', color='tab:gray')
plt.plot(fitting_values[:-1], lr_z_score_differences, label='Z-Score Difference', marker='o', color='tab:purple')
plt.plot(fitting_values[:-1], lr_accuracy_differences, label='Accuracy Difference', marker='o', color='tab:red')
plt.plot(fitting_values[:-1], lr_precision_differences, label='Precision Difference', marker='o', color='tab:pink')
plt.plot(fitting_values[:-1], lr_recall_differences, label='Recall Difference', marker='o', color='tab:orange')
plt.xlabel('Fitting Values')
plt.ylabel('Score Difference')
plt.title('Score Differences for Linear Regression')
plt.legend()
plt.grid(True)
plt.show()

RANDOM FOREST

In [ ]:
# Initialize empty lists to store the differences in scores for Random Forest
rf_mae_differences = []
rf_mse_differences = []
rf_z_score_differences = []
rf_accuracy_differences = []
rf_precision_differences = []
rf_recall_differences = []

# Loop through fitting values and calculate scores for Random Forest
for i in range(len(fitting_values) - 1):
    # Define the fitting value for this iteration
    fitting_value = fitting_values[i]

    # Split the data into training and testing sets with the current fitting value
    X_train_sampled, X_test_sampled, y_train_sampled, y_test_sampled = train_test_split(
        X, y, test_size=fitting_value, random_state=42
    )

    # Create a Random Forest model
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Fit the model to the training data
    rf_model.fit(X_train_sampled, y_train_sampled)

    # Predict the price change for the test data
    rf_predictions = rf_model.predict(X_test_sampled)

    # Define a threshold for predictions (e.g., 0.01 for a small price change)
    threshold = 0.01

    # Calculate the direction of the price change based on the threshold
    y_test_direction = np.where(y_test_sampled > threshold, 1, np.where(y_test_sampled < -threshold, -1, 0))
    predictions_direction = np.where(rf_predictions > threshold, 1, np.where(rf_predictions < -threshold, -1, 0))

    # Calculate MAE and MSE for Random Forest
    rf_mae_score = mean_absolute_error(y_test_sampled, rf_predictions)
    rf_mse_score = mean_squared_error(y_test_sampled, rf_predictions)

    # Calculate Z-score, accuracy, and precision for Random Forest
    rf_z_score = (accuracy_score(y_test_direction, predictions_direction) - 0.5) / np.sqrt(
        (0.5 * (1 - 0.5)) / len(y_test_sampled)
    )
    rf_accuracy = accuracy_score(y_test_direction, predictions_direction)
    rf_precision = precision_score(y_test_direction, predictions_direction, average='micro')  # Use binary average
    rf_recall = recall_score(y_test_direction, predictions_direction, average='micro')

    # Append the differences to the lists for Random Forest
    rf_mae_differences.append(rf_mae_score)
    rf_mse_differences.append(rf_mse_score)
    rf_z_score_differences.append(rf_z_score)
    rf_accuracy_differences.append(rf_accuracy)
    rf_precision_differences.append(rf_precision)
    rf_recall_differences.append(rf_recall)

# Create a line chart to visualize the differences for Random Forest
plt.figure(figsize=(12, 8))
plt.plot(fitting_values[:-1], rf_mae_differences, label='Random Forest MAE Difference', marker='o', color='tab:blue')
plt.plot(fitting_values[:-1], rf_mse_differences, label='Random Forest MSE Difference', marker='o', color='tab:orange')
plt.plot(fitting_values[:-1], rf_z_score_differences, label='Random Forest Z-Score Difference', marker='o', color='tab:green')
plt.plot(fitting_values[:-1], rf_accuracy_differences, label='Random Forest Accuracy Difference', marker='o', color='tab:red')
plt.plot(fitting_values[:-1], rf_precision_differences, label='Random Forest Precision Difference', marker='o', color='tab:purple')
plt.plot(fitting_values[:-1], rf_recall_differences, label='Random Forest Recall Difference', marker='o', color='tab:brown')
plt.xlabel('Fitting Values')
plt.ylabel('Score Difference')
plt.title('Score Differences for Random Forest')
plt.legend()
plt.grid(True)
plt.show()

LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Initialize empty lists to store the differences in scores for LSTM
lstm_mae_differences = []
lstm_mse_differences = []
lstm_z_score_differences = []
lstm_accuracy_differences = []
lstm_precision_differences = []
lstm_recall_differences = []

# Loop through fitting values and calculate scores for LSTM
for i in range(len(fitting_values) - 1):
    # Define the fitting value for this iteration
    fitting_value = fitting_values[i]

    # Split the data into training and testing sets with the current fitting value
    X_train_sampled, X_test_sampled, y_train_sampled, y_test_sampled = train_test_split(
        X, y, test_size=fitting_value, random_state=42
    )

    # Reshape data for LSTM (samples, time steps, features)
    X_train_lstm = X_train_sampled.values.reshape(X_train_sampled.shape[0], 1, X_train_sampled.shape[1])
    X_test_lstm = X_test_sampled.values.reshape(X_test_sampled.shape[0], 1, X_test_sampled.shape[1])

    # Create an LSTM model
    lstm_model = Sequential()
    lstm_model.add(LSTM(50, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
    lstm_model.add(Dense(1))
    lstm_model.compile(loss='mean_squared_error', optimizer='adam')

    # Fit the LSTM model to the training data
    lstm_model.fit(X_train_lstm, y_train_sampled, epochs=50, batch_size=32, verbose=0)

    # Predict the price change for the test data using the LSTM model
    lstm_predictions = lstm_model.predict(X_test_lstm).flatten()

    # Define a threshold for predictions (e.g., 0.01 for a small price change)
    threshold = 0.01

    # Calculate the direction of the price change based on the threshold
    y_test_direction = np.where(y_test_sampled > threshold, 1, np.where(y_test_sampled < -threshold, -1, 0))
    predictions_direction = np.where(lstm_predictions > threshold, 1, np.where(lstm_predictions < -threshold, -1, 0))

    # Calculate MAE and MSE for LSTM
    lstm_mae_score = mean_absolute_error(y_test_sampled, lstm_predictions)
    lstm_mse_score = mean_squared_error(y_test_sampled, lstm_predictions)

    # Calculate Z-score, accuracy, and precision for LSTM
    lstm_z_score = (accuracy_score(y_test_direction, predictions_direction) - 0.5) / np.sqrt(
        (0.5 * (1 - 0.5)) / len(y_test_sampled)
    )
    lstm_accuracy = accuracy_score(y_test_direction, predictions_direction)
    lstm_precision = precision_score(y_test_direction, predictions_direction, average='micro')  # Use binary average
    lstm_recall = recall_score(y_test_direction, predictions_direction, average='micro')

    # Append the differences to the lists for LSTM
    lstm_mae_differences.append(lstm_mae_score)
    lstm_mse_differences.append(lstm_mse_score)
    lstm_z_score_differences.append(lstm_z_score)
    lstm_accuracy_differences.append(lstm_accuracy)
    lstm_precision_differences.append(lstm_precision)
    lstm_recall_differences.append(lstm_recall)

# Create a line chart to visualize the differences for LSTM
plt.figure(figsize=(12, 8))
plt.plot(fitting_values[:-1], lstm_mae_differences, label='LSTM MAE Difference', marker='o', color='tab:blue')
plt.plot(fitting_values[:-1], lstm_mse_differences, label='LSTM MSE Difference', marker='o', color='tab:orange')
plt.plot(fitting_values[:-1], lstm_z_score_differences, label='LSTM Z-Score Difference', marker='o', color='tab:green')
plt.plot(fitting_values[:-1], lstm_accuracy_differences, label='LSTM Accuracy Difference', marker='o', color='tab:red')
plt.plot(fitting_values[:-1], lstm_precision_differences, label='LSTM Precision Difference', marker='o', color='tab:purple')
plt.plot(fitting_values[:-1], lstm_recall_differences, label='LSTM Recall Difference', marker='o', color='tab:brown')
plt.xlabel('Fitting Values')
plt.ylabel('Score Difference')
plt.title('Score Differences for LSTM')
plt.legend()
plt.grid(True)
plt.show()


**PLOTTING COMPARISSON BETWEEN MODELS**

In [ ]:
import matplotlib.pyplot as plt

# Create a line chart to visualize the differences with colorblind-friendly colors
plt.figure(figsize=(12, 8))

# Plot Linear Regression metrics
plt.plot(fitting_values[:-1], lr_mae_differences, label='Linear Regression MAE', marker='o', color='tab:cyan')
plt.plot(fitting_values[:-1], lr_mse_differences, label='Linear Regression MSE', marker='o', color='tab:gray')
plt.plot(fitting_values[:-1], lr_z_score_differences, label='Linear Regression Z-Score', marker='o', color='tab:purple')
plt.plot(fitting_values[:-1], lr_accuracy_differences, label='Linear Regression Accuracy', marker='o', color='tab:red')
plt.plot(fitting_values[:-1], lr_precision_differences, label='Linear Regression Precision', marker='o', color='tab:pink')
plt.plot(fitting_values[:-1], lr_recall_differences, label='Linear Regression Recall', marker='o', color='tab:orange')

# Plot Random Forest metrics
plt.plot(fitting_values[:-1], rf_mae_differences, label='Random Forest MAE', marker='o', linestyle='dashed', color='tab:cyan')
plt.plot(fitting_values[:-1], rf_mse_differences, label='Random Forest MSE', marker='o', linestyle='dashed', color='tab:gray')
plt.plot(fitting_values[:-1], rf_z_score_differences, label='Random Forest Z-Score', marker='o', linestyle='dashed', color='tab:purple')
plt.plot(fitting_values[:-1], rf_accuracy_differences, label='Random Forest Accuracy', marker='o', linestyle='dashed', color='tab:red')
plt.plot(fitting_values[:-1], rf_precision_differences, label='Random Forest Precision', marker='o', linestyle='dashed', color='tab:pink')
plt.plot(fitting_values[:-1], rf_recall_differences, label='Random Forest Recall', marker='o', linestyle='dashed', color='tab:orange')

# Plot LSTM metrics
plt.plot(fitting_values[:-1], lstm_mae_differences, label='LSTM MAE', marker='o', linestyle='dotted', color='tab:cyan')
plt.plot(fitting_values[:-1], lstm_mse_differences, label='LSTM MSE', marker='o', linestyle='dotted', color='tab:gray')
plt.plot(fitting_values[:-1], lstm_z_score_differences, label='LSTM Z-Score', marker='o', linestyle='dotted', color='tab:purple')
plt.plot(fitting_values[:-1], lstm_accuracy_differences, label='LSTM Accuracy', marker='o', linestyle='dotted', color='tab:red')

plt.xlabel('Fitting Values')
plt.ylabel('Score Difference')
plt.title('Score Differences for Different Models')
plt.legend()
plt.grid(True)
plt.show()


**IMPORTING AND MERGING UP TO DATE HISTORICAL DATA**

In [ ]:
from datetime import datetime

In [ ]:
# Define the ticker symbol for S&P 500 (SPY is the ETF that tracks it)
ticker_symbol = "SPY"

# Define the start and end dates for your data
start_date = "2000-12-01"
end_date =  datetime.today().strftime('%d-%m-%Y')

# Fetch historical S&P 500 data from Yahoo Finance
sp500_data_uptodate = yf.download(ticker_symbol, start=start_date, end=end_date)

In [ ]:
# Replace 'YOUR_API_KEY' with your actual FRED API key
api_key = 'YOUR API KEY HERE'

# Initialize the FRED API client
fred = Fred(api_key=api_key)

# Define the series IDs for the economic indicators you want to retrieve
inflation_series_id = 'CPIAUCSL'  # Example: Consumer Price Index for All Urban Consumers (CPI-U)
interest_rate_series_id = 'FEDFUNDS'  # Example: Effective Federal Funds Rate
labor_openings_series_id = 'JTSJOL'  # Example: Job Openings, Total Nonfarm

# Fetch economic indicator data from FRED
inflation_data_uptodate = fred.get_series(inflation_series_id,start=start_date, end=end_date)
interest_rate_data_uptodate = fred.get_series(interest_rate_series_id,start=start_date, end=end_date)
labor_openings_data_uptodate = fred.get_series(labor_openings_series_id,start=start_date, end=end_date)


In [ ]:
# Convert the economic data Series to DataFrames with appropriate column names
inflation_df_uptodate = pd.DataFrame({'Inflation_Rate': inflation_data})
interest_rate_df_uptodate = pd.DataFrame({'Interest_Rate': interest_rate_data})
labor_openings_df_uptodate = pd.DataFrame({'Labor_Openings': labor_openings_data})

# Merge the DataFrames with your S&P 500 data
merged_data_uptodate = sp500_data.join([inflation_df_uptodate, interest_rate_df_uptodate, labor_openings_df_uptodate], how='inner')

In [ ]:
# Calculate 7-day rolling mean and standard deviation of S&P 500 prices
merged_data_uptodate['SMA_7'] = merged_data_uptodate['Close'].rolling(window=7).mean()
merged_data_uptodate['STD_7'] = merged_data_uptodate['Close'].rolling(window=7).std()

# Calculate percentage change in S&P 500 prices
merged_data_uptodate['Price_Change'] = merged_data_uptodate['Close'].pct_change()

In [ ]:
merged_data_uptodate = merged_data_uptodate.iloc[6:]

In [ ]:
merged_data_uptodate.head(-1)

**RUNNING LSTM MODEL ON UP TO DATE DATA WITH 0.5 FITTING**

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define your features and target variable
X = merged_data_uptodate[['SMA_7', 'STD_7', 'Inflation_Rate', 'Interest_Rate', 'Labor_Openings']]
y = merged_data_uptodate['Price_Change']

# Spliting the data into training, validation, and test sets
# training (50%) and temporary (50%)
X_train_temp, X_temp, y_train_temp, y_temp = train_test_split(X, y, test_size=0.5, random_state=42)

# Spliting the temporary set into validation (25%) and test (25%)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the scaler on training data
X_train_lstm = scaler.fit_transform(X_train_temp)

# Transform validation and test data using the same scaler
X_validation_lstm = scaler.transform(X_validation)
X_test_lstm = scaler.transform(X_test)

# Reshape data for LSTM (samples, time steps, features)
num_features = X_train_temp.shape[1]  # Number of features
X_train_lstm = X_train_lstm.reshape(X_train_temp.shape[0], 1, num_features)
X_validation_lstm = X_validation_lstm.reshape(X_validation.shape[0], 1, num_features)
X_test_lstm = X_test_lstm.reshape(X_test.shape[0], 1, num_features)

# Create an LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Fit the LSTM model to the training data and validate on the validation set
lstm_model.fit(X_train_lstm, y_train_temp, epochs=50, validation_data=(X_validation_lstm, y_validation), callbacks=[early_stopping])

# Predict the price change for the test data using the LSTM model
lstm_predictions = lstm_model.predict(X_test_lstm).flatten()


In [ ]:
selected_features = ['Inflation_Rate',	'Interest_Rate',	'Labor_Openings',	'SMA_7',	'STD_7']
historical_data_lstm = merged_data_uptodate[selected_features].values

In [ ]:
sequence_length = 5
historical_data_lstm = np.array([historical_data_lstm[i:i+sequence_length] for i in range(len(historical_data_lstm)-sequence_length+1)])

historical_data_lstm.shape


In [ ]:
sequence_length = 5
num_features = 5

# Reshape the data
historical_data_lstm = historical_data_lstm.reshape(-1, 1, num_features)

# Now, historical_data_lstm should have the shape (num_samples, 1, 5)
lstm_predictions = lstm_model.predict(historical_data_lstm)

historical_data_lstm.shape

In [ ]:
lstm_predictions = lstm_model.predict(historical_data_lstm)

In [ ]:
# Define the date for which you want to make a prediction
target_date = '2024-06-21'

# Extract the prediction for the specific date (e.g., the last date in the sequence)
predicted_price_change = lstm_predictions[-1][0]

# Assuming you want to predict the Close price, you can add the predicted price change to the previous Close price
# Access the Z-axis value directly from the NumPy array
previous_close_price = historical_data_lstm[-1][0][0]  # Replace [0][0] with the appropriate indices for your Z-axis

predicted_close_price = previous_close_price * (1 + predicted_price_change)

# Print the predicted Close price for the future date
print(f"Predicted Close Price for {target_date}: {predicted_close_price:.2f}")
